In [4]:
! pip install hyperopt
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK

In [5]:
# Example objective function
def objective_function(params):
    # Simulate an evaluation with a random "loss"
    loss = np.random.uniform(0, 1)
    # Return a dictionary with the loss and the status
    return {'loss': loss, 'status': STATUS_OK}

# Define the space of hyperparameters to search
space = {
    'hyperparameter1': hp.uniform('hyperparameter1', 0, 1),
    'hyperparameter2': hp.choice('hyperparameter2', ['option1', 'option2']),
}

# Run the optimization
best_hyperparameters = fmin(
    fn=objective_function,  # Function to minimize
    space=space,            # Search space
    algo=tpe.suggest,       # Algorithm (TPE)
    max_evals=100           # Number of evaluations
)

print(best_hyperparameters)

100%|██████████| 100/100 [00:00<00:00, 252.45trial/s, best loss: 0.013021815325322761]
{'hyperparameter1': 0.1773098727553002, 'hyperparameter2': 1}


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from hyperopt import fmin, hp, tpe, STATUS_OK
import numpy as np

# Define a function to train and evaluate ResNet18 with given hyperparameters
def train_evaluate(params):
    # Define transforms for the data
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load CIFAR-10 dataset
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    # Define data loaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

    # Define ResNet18 model
    model = models.resnet18()

    # Set hyperparameters
    lr = params['lr']
    momentum = params['momentum']
    weight_decay = params['weight_decay']

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    # Train the model
    model.train()
    for epoch in range(5):  # Train for 5 epochs for demonstration
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return {'loss': 1 - accuracy, 'status': STATUS_OK}  # Minimize 1 - accuracy

# Define the space of hyperparameters to search
space = {
    'lr': hp.uniform('lr', 0.001, 0.1),
    'momentum': hp.uniform('momentum', 0.5, 0.9),
    'weight_decay': hp.uniform('weight_decay', 0.0001, 0.01)
}

# Run the optimization
best_hyperparameters = fmin(
    fn=train_evaluate,  # Function to minimize
    space=space,        # Search space
    algo=tpe.suggest,   # Algorithm (TPE) Bayesian Optimization
    max_evals=10        # Number of evaluations
)

print("Best Hyperparameters:", best_hyperparameters)